In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk
import spacy

In [3]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [4]:
nlp = spacy.load("en_core_web_sm")


In [5]:
def extract_entities(text):
    # Tokenize and perform named entity recognition
    tokens = word_tokenize(text)
    pos_tags = pos_tag(tokens)
    named_entities = ne_chunk(pos_tags)

    # Extract relevant entities
    entities = {}
    for chunk in named_entities:
        if hasattr(chunk, 'label'):
            entity_type = chunk.label()
            entity_value = ' '.join(c[0] for c in chunk)
            entities[entity_type] = entity_value

    return entities

In [6]:
def identify_intent(text):
    # Use spaCy for intent classification
    doc = nlp(text)

    # Simple rule-based intent classification
    if any(token.lemma_ in ["show", "display", "list"] for token in doc):
        return "SELECT"
    elif any(token.lemma_ in ["update", "change", "modify"] for token in doc):
        return "UPDATE"
    elif any(token.lemma_ in ["add", "insert", "create"] for token in doc):
        return "INSERT"
    elif any(token.lemma_ in ["delete", "remove"] for token in doc):
        return "DELETE"
    else:
        return "UNKNOWN"

In [8]:
def generate_sql_query(text):
    entities = extract_entities(text)
    intent = identify_intent(text)

    # Simple query generation based on intent and entities
    if intent == "SELECT":
        table_name = entities.get("GPE", "table_name")  # Assuming GPE (Geo-Political Entity) as table name
        return f"SELECT * FROM {table_name};"
    elif intent == "UPDATE":
        table_name = entities.get("GPE", "table_name")
        return f"UPDATE {table_name} SET column = value WHERE condition;"
    elif intent == "INSERT":
        table_name = entities.get("GPE", "table_name")
        return f"INSERT INTO {table_name} (column1, column2) VALUES (value1, value2);"
    elif intent == "DELETE":
        table_name = entities.get("GPE", "table_name")
        return f"DELETE FROM {table_name} WHERE condition;"
    else:
        return "Unable to generate SQL query."


In [9]:
text = "Show all cases in Beijing"
sql_query = generate_sql_query(text)
print(sql_query)

SELECT * FROM Beijing;
